In [1]:
#| default_exp foundation.export

In [33]:
#| export
import os
from fastcore.basics import ifnone
from fastcore.foundation import L
from fastcore.script import call_parse

from nbdev.config import get_config, add_init
from nbdev.doclinks import nbglob, _build_modidx
from nbdev.export import ExportModuleProc, black_format
from nbdev.maker import ModuleMaker
from nbdev.process import NBProcessor
from nbdev.processors import FilterDefaults

In [24]:
#| export
def nb_export(nbname, lib_path=None, procs=black_format, debug=False, mod_maker=ModuleMaker, name=None):
    """Create module(s) from notebook"""
    if lib_path is None: lib_path = get_config().lib_path
    exp = ExportModuleProc()
    extra_procs = FilterDefaults().xtra_procs()
    nb = NBProcessor(nbname, [exp]+L(procs) + extra_procs, debug=debug)
    nb.process()
    for mod,cells in exp.modules.items():
        all_cells = exp.in_all[mod]
        nm = ifnone(name, getattr(exp, 'default_exp', None) if mod=='#' else mod)
        if not nm:
            warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
                 "Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.\n"
                 "See https://nbdev.fast.ai/getting_started.html for more information.")
            return
        mm = mod_maker(dest=lib_path, name=nm, nb_path=nbname, is_new=bool(name) or mod=='#')
        mm.make(cells, all_cells, lib_path=lib_path)

In [ ]:
#| export
@call_parse
def fast_export(
    path:str=None, # Path or filename
):
    """
    Export notebooks in `path` to Python modules
    """
    if os.environ.get('IN_TEST',0): return
    files = nbglob(path=path, as_path=True).sorted('name')
    for f in files: nb_export(f)
    add_init(get_config().lib_path)
    _build_modidx()